<a href="https://colab.research.google.com/github/pradeepyes/javascript/blob/main/Weather_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Section 1: Import Libraries

In [ ]:
import requests # This library helps us to fetch data from API
import pandas as pd #for handling and analysing data
import numpy as np #for numerical operations
from sklearn.model_selection import train_test_split #to split data into training and testing sets
from sklearn.preprocessing import LabelEncoder #to convert categorical data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #models for classification and regression tasks
from sklearn.metrics import mean_squared_error #to measure the accuracy of our predictions
from datetime import datetime, timedelta #to handle date and time
import pytz


In [ ]:
API_KEY ='2d5dcce4ccb351dce73c363f965badc6'
BASE_URL ='https://api.openweathermap.org/data/2.5/' #base url for making API request


Fetch current weather Data

In [ ]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"  # construct the API request URL
    response = requests.get(url)  # send the GET request to API
    data = response.json()
    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'pressure': data['main']['pressure'],
        'wind_Gust_speed': data['wind']['speed'],
    }


Read Historical Data

In [ ]:
def read_historical_data(filename):
    df = pd.read_csv(filename)  # load csv file into DataFrame
    df = df.dropna()  # remove rows with missing values
    df = df.drop_duplicates()
    return df


Prepare Data for training


In [ ]:
def prepare_data(data):
  le = LabelEncoder() #create a LabelEncoder instance
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  #define the feature variable and target variables
  X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']] #feature variables
  y = data['RainTomorrow'] #target variable

  return X, y, le #return feture variable, target variable and the label encoder

Train Rain Pridiction Model

In [ ]:
def train_rain_model(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, y_train) #train the model
  y_pred = model.predict(X_test) #to make predictions on test set
  print("Mean Squared Error for Rain Model")
  print(mean_squared_error(y_test, y_pred))
  return model

Preapre regression data

In [ ]:
def prepare_regression_data(data, feature):
  X, y = [], [] #initialize list for feature and target values

  for i in range(len(data) - 1):
    X.append(data[feature].iloc[i])

    y.append(data[feature].iloc[i+1])

  X = np.array(X).reshape(-1, 1)
  y = np.array(y)

  return X, y

Train Regression model

In [ ]:
def train_regression_model(X, y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X, y)
  return model

Predict future

In [ ]:
def predict_future(model, current_value):
  predictions = [current_value]
  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))
    predictions.append(next_value[0])
  return predictions[1:]

Weather Analysis Function

In [ ]:
def weather_view():
    city = input('Enter any city name: ')

    # Fetch current weather
    current_weather = get_current_weather(city)

    # Debug: Print to check if current_weather is fetched correctly
    print("Debug: current_weather =", current_weather)

    if not current_weather:
        print("Error: Unable to fetch weather data.")
        return  # Exit if no data is retrieved

    # Proceed with the rest of the function
    historical_data = read_historical_data('/content/drive/MyDrive/weather.csv')

    # Prepare and train the rain prediction model
    X, y, le = prepare_data(historical_data)
    rain_model = train_rain_model(X, y)

    # Map wind direction
    wind_deg = current_weather.get('wind_gust_dir', 0) % 360

    compass_points = [
        ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75)
    ]

    compass_direction = next(
        (point for point, start, end in compass_points if start <= wind_deg < end), "N"
    )

    compass_direction_encoded = (
        le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1
    )

    # Construct current data dictionary
    current_data = {
        'MinTemp': current_weather.get('temp_min', 0),
        'MaxTemp': current_weather.get('temp_max', 0),
        'WindGustDir': compass_direction_encoded,
        'WindGustSpeed': current_weather.get('Wind_Gust_Speed', 0),
        'Humidity': current_weather.get('humidity', 0),
        'Pressure': current_weather.get('pressure', 0),
        'Temp': current_weather.get('current_temp', 0),
    }

    current_df = pd.DataFrame([current_data])

    # Rain prediction
    rain_prediction = rain_model.predict(current_df)[0]

    # Prepare regression models
    X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

    temp_model = train_regression_model(X_temp, y_temp)
    hum_model = train_regression_model(X_hum, y_hum)

    # Predict future values
    future_temp = predict_future(temp_model, current_weather.get('temp_min', 0))
    future_humidity = predict_future(hum_model, current_weather.get('humidity', 0))

    # Time setup for predictions
    timezone = pytz.timezone('Asia/Kolkata')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    # Display results
    print(f"City: {city}, {current_weather.get('country', 'Unknown')}")
    print(f"Current Temperature: {current_weather.get('current_temp', 0)}")
    print(f"Feels Like: {current_weather.get('feels_like', 0)}")
    print(f"Minimum Temperature: {current_weather.get('temp_min', 0)}°C")
    print(f"Maximum Temperature: {current_weather.get('temp_max', 0)}°C")
    print(f"Humidity: {current_weather.get('humidity', 0)}%")
    print(f"Weather Prediction: {current_weather.get('description', 'Unknown')}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

    print("\nFuture Temperature Predictions:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\nFuture Humidity Predictions:")
    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")


# Run the function
weather_view()


Enter any city name: Davanagere
Debug: current_weather = {'city': 'Davanagere', 'current_temp': 37, 'feels_like': 34, 'temp_min': 37, 'temp_max': 37, 'humidity': 14, 'description': 'broken clouds', 'country': 'IN', 'wind_gust_dir': 77, 'pressure': 1006, 'wind_Gust_speed': 4.09}
Mean Squared Error for Rain Model
0.1506849315068493
City: Davanagere, IN
Current Temperature: 37
Feels Like: 34
Minimum Temperature: 37°C
Maximum Temperature: 37°C
Humidity: 14%
Weather Prediction: broken clouds
Rain Prediction: No

Future Temperature Predictions:
17:00: 33.1°C
18:00: 25.1°C
19:00: 20.1°C
20:00: 16.6°C
21:00: 14.4°C

Future Humidity Predictions:
17:00: 44.8%
18:00: 46.2%
19:00: 48.2%
20:00: 53.7%
21:00: 58.1%
